# xDSL-MLIR interoperation tutorial

This tutorial aims to showcase a simple pipeline of actions to unlock MLIR optimisations when lowering from xDSL.
This tutorial can help users getting familiar with the xDSL-MLIR interoperation. We will start from a higher level of xDSL abstraction, lower to MLIR generic format, apply an optimisation and the return to xDSL-land.

# Problem setup

We start by writing a simple example consisting of adding integers. 
We are writing this example using constructs that are supported in xDSL.

We create 4 integers, namely a, b, c, d.
Then we just accumulate by the simple following pseudocode and print the result:


```bash
a = 1
b = 2
c = a + b
d = a + b
e = c + d
print(e)
```


In [1]:
from xdsl.dialects.arith import Addi, Constant
from xdsl.dialects.builtin import i32, ModuleOp, IntegerAttr
from xdsl.dialects.vector import Print
from xdsl.ir import Region, Block

# Define two integer constants
a = Constant.from_attr(IntegerAttr.from_int_and_width(1, 32), i32)
b = Constant.from_attr(IntegerAttr.from_int_and_width(2, 32), i32)

# Operations on these constants
c = Addi.get(a, b)
d = Addi.get(a, b)
e = Addi.get(c, d)
f = Print.get(e)

# Create Block from operations and Region from blocks
block0 = Block.from_ops([a, b, c, d, e, f])
region0 = Region.from_block_list([block0])

# Create an Operation from the region
op = ModuleOp.from_region_or_ops(region0)

Using xDSLs printer we can print this operation.
For convenience we provide a file called `source.xdsl` with the code printed below

In [2]:
from xdsl.printer import Printer

# Print in xdsl format
printer = Printer()
printer.print(op)

builtin.module() {
  %0 : !i32 = arith.constant() ["value" = 1 : !i32]
  %1 : !i32 = arith.constant() ["value" = 2 : !i32]
  %2 : !i32 = arith.addi(%0 : !i32, %1 : !i32)
  %3 : !i32 = arith.addi(%0 : !i32, %1 : !i32)
  %4 : !i32 = arith.addi(%2 : !i32, %3 : !i32)
  vector.print(%4 : !i32)
}


In [3]:
# Cross-check file content
!cat source.xdsl

builtin.module() {
  %0 : !i32 = arith.constant() ["value" = 1 : !i32]
  %1 : !i32 = arith.constant() ["value" = 2 : !i32]
  %2 : !i32 = arith.addi(%0 : !i32, %1 : !i32)
  %3 : !i32 = arith.addi(%0 : !i32, %1 : !i32)
  %4 : !i32 = arith.addi(%2 : !i32, %3 : !i32)
  vector.print(%4 : !i32)
}


Then we are using the `xdsl-opt` tool provided from the repository to emit .mlir code from the .xdsl input.
You can see in the following cell that `xdsl-opt` can be used in various contexts such as translating back and forth from/to xdsl/mlir. 

In [4]:
# xdsl-opt help
!./../xdsl/tools/xdsl-opt -h

usage: xdsl-opt [-h] [-t {xdsl,irdl,mlir}] [-f {xdsl,mlir}] [--disable-verify]
                [-o OUTPUT_FILE] [-p PASSES] [--print-between-passes]
                [--verify-diagnostics] [--parsing-diagnostics]
                [--allow-unregistered-ops]
                [input_file]

xDSL modular optimizer driver

positional arguments:
  input_file            path to input file

options:
  -h, --help            show this help message and exit
  -t {xdsl,irdl,mlir}, --target {xdsl,irdl,mlir}
                        target
  -f {xdsl,mlir}, --frontend {xdsl,mlir}
                        Frontend to be used for the input. If not set, the
                        xdsl frontend or the one for the file extension is
                        used.
  --disable-verify
  -o OUTPUT_FILE, --output-file OUTPUT_FILE
                        path to output file
  -p PASSES, --passes PASSES
                        Delimited list of passes. Available passes are: lower-
                        mpi,convert-s

In [5]:
# Checkout the mlir output in generic form
!./../xdsl/tools/xdsl-opt -f xdsl -t mlir -o out.mlir source.xdsl
!cat out.mlir

"builtin.module"() ({
  %0 = "arith.constant"() {"value" = 1 : i32} : () -> i32
  %1 = "arith.constant"() {"value" = 2 : i32} : () -> i32
  %2 = "arith.addi"(%0, %1) : (i32, i32) -> i32
  %3 = "arith.addi"(%0, %1) : (i32, i32) -> i32
  %4 = "arith.addi"(%2, %3) : (i32, i32) -> i32
  "vector.print"(%4) : (i32) -> ()
}) : () -> ()



Now lets try to benefit from some mlir optimisation.
For this example we will use the [Common subexpression elimination](https://en.wikipedia.org/wiki/Common_subexpression_elimination).

See some documentation here: [mlir.llvm CSE docs](https://mlir.llvm.org/docs/Passes/#-cse-eliminate-common-sub-expressions)

Assuming you have already mlir-opt installed in your machine you can apply the CSE optimisation using the folloing command:

In [6]:
!mlir-opt out.mlir -cse --mlir-print-op-generic -o opt-out.mlir
!cat opt-out.mlir

"builtin.module"() ({
  %0 = "arith.constant"() {value = 1 : i32} : () -> i32
  %1 = "arith.constant"() {value = 2 : i32} : () -> i32
  %2 = "arith.addi"(%0, %1) : (i32, i32) -> i32
  %3 = "arith.addi"(%2, %2) : (i32, i32) -> i32
  "vector.print"(%3) : (i32) -> ()
}) : () -> ()



We can clearly see in the optimised output that after CSE we do not need to calculate:

```
"arith.addi"(%0, %1) : (i32, i32) -> i32
```

twice! Now can we back to xDSL? Yes we can!

In [7]:
!./../xdsl/tools/xdsl-opt opt-out.mlir -f mlir -t xdsl -o ret.xdsl
!cat ret.xdsl

builtin.module() {
  %0 : !i32 = arith.constant() ["value" = 1 : !i32]
  %1 : !i32 = arith.constant() ["value" = 2 : !i32]
  %2 : !i32 = arith.addi(%0 : !i32, %1 : !i32)
  %3 : !i32 = arith.addi(%2 : !i32, %2 : !i32)
  vector.print(%3 : !i32)
}

